## 8.1

In [ ]:
import pandas as pd
ramen = pd.read_csv('ramen-ratings.csv', na_values = {'Stars': ['Unrated']})
ramen.drop('Review #', axis = 1, inplace = True)
ramen.loc[ramen.Country == 'United States', 'Country'] = 'USA'

median_ratings = ramen.groupby(by = ['Brand', 'Style']).median()

In [ ]:
ramen.head()

## 8.2


In [ ]:
import pandas as pd
import numpy as np
name = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
         'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']

cleveland = pd.read_csv('processed.cleveland.data', na_values = '?', names = name)

cleveland.loc[cleveland.num != 0, 'num'] = 1
bins = [0, 40, 50, 60, 70, 100]
cut = pd.cut(cleveland.age, bins)
groups = cleveland.groupby(by = [cut, 'sex', 'restecg'])

def proportion(array):
    return sum(array == 1) / len(array)

names = ['age_group', ]
disease_percent = groups['num'].agg([('n', 'count'), ('disease_pct', proportion)]).reset_index()
disease_percent = disease_percent.rename(columns = {'age': 'age_group'})


In [ ]:
cleveland.head()

In [ ]:
cleveland.loc[cleveland.num != 0, 'num'] = 1
bins = [0, 40, 50, 60, 70, 100]
cut = pd.cut(cleveland.age, bins)
groups = cleveland.groupby(by = [cut, 'sex', 'restecg'])

def proportion(array):
    return sum(array == 1) / len(array)

names = ['age_group', ]
disease_percent = groups['num'].agg([('n', 'count'), ('disease_pct', proportion)]).reset_index()
disease_percent = disease_percent.rename(columns = {'age': 'age_group'})

## 8.3


In [34]:
import re
import pandas as pd
import numpy as np
phone_numbers = pd.read_csv('phone-numbers.csv')
pattern = '^[\s(]*(\d{3})[-)\s]*(\d{3})[-\s]*(\d{4})\s*$'
combined = phone_numbers.phone.str.extract(pattern).astype(str)
phone_numbers['phone'] = combined[[0,1,2]].agg('-'.join, axis = 1)
phone_numbers.replace('nan-nan-nan', np.nan, inplace = True)


In [32]:
pattern = r'[\s(]*\d{3}[\s)-]*\d{3}[\s-]*\d{4}[\s]*$'
match = phone_numbers['phone'].str.match(pattern)
phone_numbers.loc[match == False, 'phone'] = np.nan

def reformat(row):
    row = str(row)
    res = ''
    if row == 'nan':
        return np.nan
    
    for c in row:
        if c.isdigit():
            res += c
    res = res[:3] + '-' + res[3:6] + '-' + res[6:]
    return res 

phone_numbers['phone'] = phone_numbers['phone'].apply(reformat)


## 8.4


In [7]:
import pandas as pd 
address = pd.read_table('address.csv', names = ['text'], lineterminator = '\\')
address['text'] = address['text'].str.replace('\n', '+')
pattern = r'\+?[\d]+.*[\s,]+[a-zA-Z]{2}[,\s]+\d'
match = address['text'].str.match(pattern)
address = address.loc[match]

def reformat(row):
    row = str(row)
    if row[0] == '+': row = row[1:]
    row = row.replace(',', '').split('+')
    city_state_zipcode = row[-1].split()
    street, city, state, zipcode = row[0], ' '.join(city_state_zipcode[:-2]), city_state_zipcode[-2].upper(), city_state_zipcode[-1]
    apt = row[1] if len(row) == 3 else ''
    row = [street, apt, city, state + ' ' + zipcode] if apt else [street, city, state + ' ' + zipcode]
    row = ', '.join(row)
    row = ' '.join([x.capitalize()[:1] + x[1:] for x in row.split()])

    return row

address['text'] = address['text'].apply(reformat)
address

,text
0,"123 Main St., Apt 121, Champaign, IL 61820-4319"
1,"15 23rd St, Urbana, IL 60181"
3,"320 Vanderbilt Way, Santa Clara, CA 95050"
4,"109 S. Forest Rd., Columbus, OH 43210-2943"


In [30]:
import pandas as pd 
import re
address = pd.read_table('address.csv', names = ['text'], lineterminator = '\\')
pattern = '^[\n]?([\d ]+[A-Z\s.]+)\n([\w ]+[\n])?([A-Z\s]+)[,\s]+([A-Z]+)[,\s]+([-\d]+)$'
separate = address.text.str.extract(pattern, flags=re.IGNORECASE).dropna(how='all').astype(str)
separate = separate.applymap(lambda row: row.title())
separate[3] = separate[3].apply(lambda row: row.upper())
separate[3] = separate[[3, 4]].agg(' '.join, axis = 1)
separate = separate.drop([4], axis = 1)
address['text'] = separate.agg(', '.join, axis = 1)
address.dropna(inplace = True)

def reformat(row):
    row = row.replace('\n', '').replace('Nan, ', '')
    return row

address['text'] = address['text'].apply(reformat)
address

,text
0,"123 Main St., Apt 121, Champaign, IL 61820-4319"
1,"15 23Rd St, Urbana, IL 60181"
3,"320 Vanderbilt Way, Santa Clara, CA 95050"
4,"109 S. Forest Rd., Columbus, OH 43210-2943"


## 8.5


In [ ]:
import pandas as pd
ramen = pd.read_csv('ramen-ratings.csv', na_values = {'Stars': ['Unrated']})
ramen.drop('Review #', axis = 1, inplace = True)
ramen.loc[ramen.Country == 'United States', 'Country'] = 'USA'

pattern = '2016\s#(\d+)'
ramen['sort'] = ramen['Top Ten'].str.extract(pattern)
rank_2016 = ramen.loc[ramen.sort.notna()]
rank_2016 = rank_2016.astype({'sort': int}).sort_values(by = 'sort')
rank_2016.drop('sort', axis = 1, inplace = True)
